In [1]:
import pandas as pd
from fuzzywuzzy import process
import requests
from bs4 import BeautifulSoup as bs
import urllib
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"}

In [15]:
full = pd.read_csv('input_csvs/FAA_AIDS_full.csv')
codes = pd.read_csv('input_csvs/FAA_AIDS_airports.csv')

In [5]:
guesses = {}
not_found = []
count = 1
for n in input_names_copy:
    url_name = urllib.parse.quote(n)
    request_str = 'https://www.iata.org/en/publications/directories/code-search/?airport.search=' + url_name
    r = requests.get(request_str, headers=headers)
    if r.status_code != 200:
        print('being blocked...')
        break
    soup = bs(r.text)
    try:
        table = soup.findAll('table', attrs={'class':'datatable'})[1]
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])
            break
        code = data[0][3]
        clean_name = data[0][2]
        guesses[n] = (code, clean_name)
    except:
        not_found.append(n)
    print(count)
    count += 1

NameError: name 'input_names_copy' is not defined

In [ ]:
guesses

In [144]:
codes
guesses_mapping = []
for airport in list(codes['eventairport']):
    try:
        guesses_mapping.append(guesses[airport])
    except:
        guesses_mapping.append('')

In [269]:
guesses_mapping

[('BKD', 'Stephens County'),
 ('FBK', 'Ladd AAF'),
 ('MFW', 'Magaruque Island'),
 ('EPA', 'El Palomar'),
 ('TTD', 'Troutdale'),
 ('HNL', 'Daniel K. Inouye Intl'),
 ('HSB', 'Raleigh'),
 ('LAF', 'Purdue University'),
 ('TED', 'Thisted'),
 '',
 ('UMZ', 'Intermountain Muni'),
 ('PWK', 'Executive'),
 ('UMZ', 'Intermountain Muni'),
 ('MBS', 'MBS Intl'),
 ('UMZ', 'Intermountain Muni'),
 '',
 ('SEE', 'Gillespie Field'),
 ('GRM', 'Cook County'),
 ('SEE', 'Gillespie Field'),
 ('ATY', 'Regional'),
 '',
 ('UMZ', 'Intermountain Muni'),
 ('MBS', 'MBS Intl'),
 ('SAQ', 'San Andros'),
 ('RBD', 'Executive'),
 ('TED', 'Thisted'),
 ('UMZ', 'Intermountain Muni'),
 '',
 ('TDW', 'Tradewind'),
 '',
 ('ISW', 'South Wood County'),
 '',
 ('MDD', 'Airpark'),
 ('LCK', 'Rickenbacker Intl'),
 ('CSV', 'Memorial-Whitson Field'),
 ('OAK', 'Metropolitan Area'),
 ('TMB', 'Miami Executive'),
 ('JFK', 'John F Kennedy Intl'),
 ('ORD', "O'Hare International"),
 ('MBS', 'MBS Intl'),
 ('RFD', 'Chicago/Rockford Intl'),
 ('MCI',

In [147]:
codes['iata_code'] = guesses_mapping

In [259]:
guesses_mapping = []
full_airports = list(full['eventairport'])
for airport in full_airports:
    try:
        guesses_mapping.append(guesses[airport])
    except:
        guesses_mapping.append('')

In [271]:
guesses_mapping
iata_code = []
clean_name = []
for g in guesses_mapping:
    if g == '':
        iata_code.append('')
        clean_name.append('')
    else:
        iata_code.append(g[0])
        clean_name.append(g[1])

In [282]:
full['iata_code'] = iata_code
full['clean_name'] = clean_name

In [283]:
full

,aidsreportnumber,localeventdate,eventcity,eventstate,eventairport,eventtype,aircraftdamage,flightphase,aircraftmake,aircraftmodel,...,aircraftenginemodel,enginegroupcode,nbrofengines,piccertificatetype,picflighttimetotalhrs,picflighttimetotalmakemodel,v26,v27,iata_code,clean_name
0,19780101000019I,01-JAN-78,WAHPETON,ND,BRECKENRIDGE,INCIDENT,MINOR,ROLL-OUT (FIXED WING),CESSNA,CE-182,...,NaN,NaN,1.0,PRIVATE PILOT,245.0,136.0,0.0,0.0,BKD,Stephens County
1,19780101000029I,01-JAN-78,FAIRBANKS,AK,FAIRBANKS INTL,INCIDENT,MINOR,ROLL-OUT (FIXED WING),PIPER,PA-18,...,NaN,NaN,1.0,STUDENT,200.0,2.0,0.0,0.0,FBK,Ladd AAF
2,19780101000039I,01-JAN-78,BRUNSWICK,GA,JEKYLL ISLAND,INCIDENT,NaN,NORMAL CRUISE,BEECH,BE-35,...,NaN,NaN,1.0,PRIVATE PILOT,NaN,0.0,0.0,0.0,MFW,Magaruque Island
3,19780101000049I,01-JAN-78,CARLSBAD,CA,MC CLELLAN-PALOMAR,INCIDENT,MINOR,LEVEL OFF TOUCHDOWN,CESSNA,CE-310,...,NaN,NaN,2.0,COMMERCIAL PILOT FLIGHT INSTRUCTOR,2000.0,400.0,0.0,0.0,EPA,El Palomar
4,19780101000059I,01-JAN-78,TROUTDALE,OR,TROUTDALE MUNI,INCIDENT,MINOR,"GROUND TAXI, OTHER AIRPLANE",CESSNA,CE-172,...,NaN,NaN,1.0,COMMERCIAL PILOT,300.0,130.0,0.0,0.0,TTD,Troutdale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103588,20181230021799I,30-DEC-18,COLUMBIA,SC,COLUMBIA METROPOLITAN,INCIDENT,MINOR,LANDING: APPROACH,CESSNA,CE-210,...,TSIO-520 SER,NaN,1.0,COMMERCIAL PILOT,1103.0,648.0,15.0,NaN,CAE,Metropolitan Apt
103589,20181230021819I,30-DEC-18,ORLANDO,FL,EXECUTIVE,INCIDENT,MINOR,TAKEOFF: CLIMB OUT,CESSNA,CE-560,...,PW545 SER,NaN,2.0,NaN,NaN,NaN,NaN,NaN,RBD,Executive
103590,20181230021829I,30-DEC-18,ALEXANDRIA,LA,ALEXANDRIA INTL,INCIDENT,SUBSTANTIAL,LANDING: TOUCHDOWN,EMBRAER,ERJ-145,...,AE3007 SER,NaN,NaN,AIRLINE TRANSPORT,17135.0,1828.0,NaN,NaN,AXL,Alexandria Station
103591,20181230021839I,30-DEC-18,SAN DIEGO,CA,MONTGOMERY FIELD,INCIDENT,MINOR,LANDING: ROLLOUT,PIPER,PA-46,...,TI0-540 SER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,


In [284]:
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
         'OCT', 'NOV', 'DEC']

In [285]:
years_raw = [str(i%100) for i in range(1978, 2021)]
years = []
for y in years_raw:
    if len(y) == 1:
        years.append('0' + y)
    else:
        years.append(y)
month_year = []
for y in years:
    for m in months:
        month_year.append(m + '-' + y)
len(month_year)

516

In [286]:
set(full['iata_code'])

{'',
 'ARG',
 'JOE',
 'SAD',
 'PQS',
 'CDB',
 'DTN',
 'MFV',
 'ROG',
 'SZS',
 'HIO',
 'BIH',
 'KPN',
 'CKE',
 'GHK',
 'MAR',
 'WKR',
 'ADR',
 'GMT',
 'UIQ',
 'CAX',
 'MGC',
 'SLC',
 'VLI',
 'CFH',
 'ONO',
 'SAS',
 'MQW',
 'DWH',
 'BAJ',
 'GDL',
 'CEU',
 'GGW',
 'MER',
 'OCE',
 'RFD',
 'IMM',
 'ABY',
 'TAM',
 'BUF',
 'ABR',
 'DAB',
 'PUG',
 'SJA',
 'KPC',
 'JVL',
 'SHN',
 'DET',
 'NRB',
 'HVN',
 'MRU',
 'KAY',
 'TNU',
 'RIJ',
 'BFK',
 'JMS',
 'CIG',
 'JSI',
 'BHM',
 'ISW',
 'SHR',
 'EKI',
 'AUG',
 'ECG',
 'EAA',
 'TKA',
 'EIL',
 'ESC',
 'HEE',
 'OCF',
 'YAR',
 'BYL',
 'NUQ',
 'OAK',
 'BIX',
 'RUI',
 'KEE',
 'FKL',
 'JNU',
 'KEN',
 'MBS',
 'BFR',
 'CKD',
 'GTG',
 'EXT',
 'FLV',
 'OAG',
 'MQT',
 'CUT',
 'MMT',
 'CBF',
 'MFI',
 'YYL',
 'TLD',
 'TOI',
 'LCK',
 'CNK',
 'KTP',
 'SSW',
 'CDG',
 'PSE',
 'YWS',
 'BNW',
 'WWD',
 'OLY',
 'ODM',
 'EWB',
 'LFI',
 'UNU',
 'CRS',
 'OCA',
 'HKY',
 'DUQ',
 'HQM',
 'CRX',
 'STJ',
 'IDG',
 'MHV',
 'HUM',
 'OXD',
 'FBY',
 'CNM',
 'EFK',
 'OMK',
 'GBR',
 'F

In [287]:
full_iata_codes = set(full['iata_code'])
set_clean_names = []
for c in full_iata_codes:
    set_clean_names.append(clean_name[iata_code.index(c)])

In [288]:
full_iata_codes

{'',
 'ARG',
 'JOE',
 'SAD',
 'PQS',
 'CDB',
 'DTN',
 'MFV',
 'ROG',
 'SZS',
 'HIO',
 'BIH',
 'KPN',
 'CKE',
 'GHK',
 'MAR',
 'WKR',
 'ADR',
 'GMT',
 'UIQ',
 'CAX',
 'MGC',
 'SLC',
 'VLI',
 'CFH',
 'ONO',
 'SAS',
 'MQW',
 'DWH',
 'BAJ',
 'GDL',
 'CEU',
 'GGW',
 'MER',
 'OCE',
 'RFD',
 'IMM',
 'ABY',
 'TAM',
 'BUF',
 'ABR',
 'DAB',
 'PUG',
 'SJA',
 'KPC',
 'JVL',
 'SHN',
 'DET',
 'NRB',
 'HVN',
 'MRU',
 'KAY',
 'TNU',
 'RIJ',
 'BFK',
 'JMS',
 'CIG',
 'JSI',
 'BHM',
 'ISW',
 'SHR',
 'EKI',
 'AUG',
 'ECG',
 'EAA',
 'TKA',
 'EIL',
 'ESC',
 'HEE',
 'OCF',
 'YAR',
 'BYL',
 'NUQ',
 'OAK',
 'BIX',
 'RUI',
 'KEE',
 'FKL',
 'JNU',
 'KEN',
 'MBS',
 'BFR',
 'CKD',
 'GTG',
 'EXT',
 'FLV',
 'OAG',
 'MQT',
 'CUT',
 'MMT',
 'CBF',
 'MFI',
 'YYL',
 'TLD',
 'TOI',
 'LCK',
 'CNK',
 'KTP',
 'SSW',
 'CDG',
 'PSE',
 'YWS',
 'BNW',
 'WWD',
 'OLY',
 'ODM',
 'EWB',
 'LFI',
 'UNU',
 'CRS',
 'OCA',
 'HKY',
 'DUQ',
 'HQM',
 'CRX',
 'STJ',
 'IDG',
 'MHV',
 'HUM',
 'OXD',
 'FBY',
 'CNM',
 'EFK',
 'OMK',
 'GBR',
 'F

In [289]:
set_clean_names

['',
 'Regional',
 'Joensuu',
 'Regional',
 'Pilot Station',
 'Cold Bay',
 'Downtown Apt',
 'Accomack County',
 'Municipal - Carter Fld',
 "Ryan's Creek",
 'Hillsboro',
 'Eastern Sierra Rgnl',
 'Kipnuk',
 'Lampson Field',
 'Gahcho Kue',
 'La Chinita Intl',
 "Walker's Cay",
 'Robert F. Swinnie',
 'Air Station',
 'Quoin Hill',
 'Airport',
 'Municipal',
 'International',
 'Port Vila',
 'Clifton Hills',
 'Municipal',
 'Salton Sea',
 'Telfair-Wheeler',
 'David Wayne Hooks Mem.',
 'Bali',
 'Miguel Hidalgo Intl',
 'Oconee County Regional',
 'International',
 'Castle',
 'Ocean City Municipal',
 'Chicago/Rockford Intl',
 'Regional',
 'Southwest Georgia Rgnl',
 'F. Javier Mina Intl',
 'Niagara International',
 'Regional',
 'International',
 'Port Augusta',
 'San Juan',
 'Coast Guard Station',
 'Southern Wisconsin Rgn',
 'Sanderson Field',
 'Coleman A. Young Muni',
 'Naval Station',
 'Tweed',
 'Sir S. Ramgoolam Intl',
 'Wakaya Island',
 'Municipal',
 'Juan Simons Vela',
 'Aurora/Buckley AFB',
 'R

In [290]:
column_entries = [[] for i in range(len(month_year))]
for code in full_iata_codes:
    chunk = full.loc[full['iata_code'] == code]
    localeventdate = list(chunk['localeventdate'])
    localeventdate = [i[3:] for i in localeventdate]
    for lst in column_entries:
        lst.append(0)
    for d in localeventdate:
        column_entries[month_year.index(d)][len(column_entries[month_year.index(d)]) - 1] += 1
        
        

In [291]:
faa_incidents_df = pd.DataFrame({'iata_code': list(full_iata_codes), 
                                 'clean_name': set_clean_names})

In [292]:
for i in range(len(column_entries)):
    faa_incidents_df[month_year[i]] = column_entries[i]

In [294]:
faa_incidents_df.to_csv('faa_incidents.csv')

In [295]:
full.to_csv('FAA_AIDS_full.csv')

In [296]:
faa_incidents_df

,iata_code,clean_name,JAN-78,FEB-78,MAR-78,APR-78,MAY-78,JUN-78,JUL-78,AUG-78,...,MAR-20,APR-20,MAY-20,JUN-20,JUL-20,AUG-20,SEP-20,OCT-20,NOV-20,DEC-20
0,,,95,74,129,123,134,144,139,152,...,0,0,0,0,0,0,0,0,0,0
1,ARG,Regional,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,JOE,Joensuu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SAD,Regional,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PQS,Pilot Station,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,COE,Pappy Boyington Field,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2147,DUC,Halliburton Field,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2148,KDK,Municipal,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2149,RDU,International,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
url_name = urllib.parse.quote(n)
request_str = 'https://www.iata.org/en/publications/directories/code-search/?airport.search=' + url_name
r = requests.get(request_str, headers=headers)

NameError: name 'n' is not defined

In [7]:
all_airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
223,4650,03N,small_airport,Utirik Airport,11.222000,169.852005,4.0,OC,MH,MH-UTI,Utirik Island,yes,K03N,UTK,03N,NaN,https://en.wikipedia.org/wiki/Utirik_Airport,NaN
440,6924,07FA,small_airport,Ocean Reef Club Airport,25.325399,-80.274803,8.0,NaN,US,US-FL,Key Largo,no,07FA,OCA,07FA,https://www.oceanreef.com/community/private-ai...,https://en.wikipedia.org/wiki/Ocean_Reef_Club_...,NaN
595,7063,0AK,small_airport,Pilot Station Airport,61.934601,-162.899994,305.0,NaN,US,US-AK,Pilot Station,yes,NaN,PQS,0AK,NaN,https://en.wikipedia.org/wiki/Pilot_Station_Ai...,NaN
674,7139,0CO2,small_airport,Crested Butte Airpark,38.851918,-106.928341,8980.0,NaN,US,US-CO,Crested Butte,no,0CO2,CSE,0CO2,NaN,NaN,Buckhorn Ranch
1089,7545,0TE7,small_airport,LBJ Ranch Airport,30.251801,-98.622498,1515.0,NaN,US,US-TX,Johnson City,no,0TE7,JCY,0TE7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56168,27242,ZYTL,large_airport,Zhoushuizi Airport,38.965698,121.539001,107.0,AS,CN,CN-21,Dalian,yes,ZYTL,DLC,NaN,NaN,https://en.wikipedia.org/wiki/Dalian_Zhoushuiz...,Dalian Air Base
56169,307013,ZYTN,medium_airport,Tonghua Sanyuanpu Airport,42.253889,125.703333,1200.0,AS,CN,CN-22,Tonghua,yes,ZYTN,TNH,NaN,NaN,https://en.wikipedia.org/wiki/Tonghua_Sanyuanp...,Tonghua Liuhe Airport
56170,27243,ZYTX,large_airport,Taoxian Airport,41.639801,123.483002,198.0,AS,CN,CN-21,Shenyang,yes,ZYTX,SHE,NaN,NaN,https://en.wikipedia.org/wiki/Shenyang_Taoxian...,NaN
56171,27244,ZYYJ,medium_airport,Yanji Chaoyangchuan Airport,42.882801,129.451004,624.0,AS,CN,CN-22,Yanji,yes,ZYYJ,YNJ,NaN,NaN,https://en.wikipedia.org/wiki/Yanji_Chaoyangch...,NaN


In [13]:
full

,aidsreportnumber,localeventdate,eventcity,eventstate,eventairport,eventtype,aircraftdamage,flightphase,aircraftmake,aircraftmodel,...,totalinjuries,aircraftenginemake,aircraftenginemodel,enginegroupcode,nbrofengines,piccertificatetype,picflighttimetotalhrs,picflighttimetotalmakemodel,v26,v27
0,19780101000019I,01-JAN-78,WAHPETON,ND,BRECKENRIDGE,INCIDENT,MINOR,ROLL-OUT (FIXED WING),CESSNA,CE-182,...,0,NaN,NaN,NaN,1.0,PRIVATE PILOT,245.0,136.0,0.0,0.0
1,19780101000029I,01-JAN-78,FAIRBANKS,AK,FAIRBANKS INTL,INCIDENT,MINOR,ROLL-OUT (FIXED WING),PIPER,PA-18,...,0,NaN,NaN,NaN,1.0,STUDENT,200.0,2.0,0.0,0.0
2,19780101000039I,01-JAN-78,BRUNSWICK,GA,JEKYLL ISLAND,INCIDENT,NaN,NORMAL CRUISE,BEECH,BE-35,...,0,NaN,NaN,NaN,1.0,PRIVATE PILOT,NaN,0.0,0.0,0.0
3,19780101000049I,01-JAN-78,CARLSBAD,CA,MC CLELLAN-PALOMAR,INCIDENT,MINOR,LEVEL OFF TOUCHDOWN,CESSNA,CE-310,...,0,NaN,NaN,NaN,2.0,COMMERCIAL PILOT FLIGHT INSTRUCTOR,2000.0,400.0,0.0,0.0
4,19780101000059I,01-JAN-78,TROUTDALE,OR,TROUTDALE MUNI,INCIDENT,MINOR,"GROUND TAXI, OTHER AIRPLANE",CESSNA,CE-172,...,0,NaN,NaN,NaN,1.0,COMMERCIAL PILOT,300.0,130.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103588,20181230021799I,30-DEC-18,COLUMBIA,SC,COLUMBIA METROPOLITAN,INCIDENT,MINOR,LANDING: APPROACH,CESSNA,CE-210,...,0,CONT M,TSIO-520 SER,NaN,1.0,COMMERCIAL PILOT,1103.0,648.0,15.0,NaN
103589,20181230021819I,30-DEC-18,ORLANDO,FL,EXECUTIVE,INCIDENT,MINOR,TAKEOFF: CLIMB OUT,CESSNA,CE-560,...,0,P&W CA,PW545 SER,NaN,2.0,NaN,NaN,NaN,NaN,NaN
103590,20181230021829I,30-DEC-18,ALEXANDRIA,LA,ALEXANDRIA INTL,INCIDENT,SUBSTANTIAL,LANDING: TOUCHDOWN,EMBRAER,ERJ-145,...,0,ROLLS-,AE3007 SER,NaN,NaN,AIRLINE TRANSPORT,17135.0,1828.0,NaN,NaN
103591,20181230021839I,30-DEC-18,SAN DIEGO,CA,MONTGOMERY FIELD,INCIDENT,MINOR,LANDING: ROLLOUT,PIPER,PA-46,...,0,LYCOMI,TI0-540 SER,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
codes

,eventairport,Freq
0,NaN,18945
1,"""BATON ROUGE METROPOLITAN, RYAN FIELD""",1
2,"""WELLSVILLE MUNI ARPT,TARANTINE FLD""",2
3,7-H SKEET CLUB INC FIELD,1
4,85TH AVENUE AIRSTRIP,1
...,...,...
6465,ZEPHYRILLS MUNI,1
6466,ZIMMERLE,1
6467,ZITONE,1
6468,ZORTMAN,1


In [21]:
set(full['eventairport'])

{nan,
 'SLIDELL',
 'SLOBOVIA OUTERNATIONAL',
 'CALHAN',
 'DANIEL',
 'FREEMAN FIELD',
 'LACROSSE MUNI',
 'SANDY CREEK AIRPARK',
 "EAGLE'S NEST",
 'FLYING V',
 'HOLMES COUNTY',
 'BRACKETT FIELD',
 'CANTWELL',
 'DEAN MEMORIAL',
 'MINNEAPOLIS',
 'SITKA ROCKY GUTIERREZ',
 'SISKIYOU COUNTY',
 'INDIANAPOLIS INTL',
 'SEASIDE MUNI',
 'BULLHEAD CITY MUNI',
 'OTTUMWA RGNL',
 'GLADWIN ZETTEL MEMORIAL',
 'EAST HAMPTON',
 'STATESBORO-BULLOCH COUNTY',
 "STARK'S TWIN OAKS AIRPARK",
 'SANTA ROSA ROUTE 66',
 'WEST MIDDLESEX',
 'MILWAUKEE MUNI',
 'SALEM MEMORIAL',
 'CHARLES B. WHEELER DOWNTOWN',
 'HUNTINGDON MUNI',
 'LAWRENCE J TIMMERMAN',
 'PAUMA VALLEY AIR PARK',
 'FINLEY RANCH',
 'OZONA MUNI',
 'VISTA FIELD',
 'PELICAN',
 'LOVELAND MUNI',
 'SIOUX CITY MUNI',
 'HUNTINGBURG',
 'SHELBY COMMUNITY',
 'ZELIENOPLE MUNI',
 'RALPH WENZ FIELD',
 'WILLIAMSPORT AIRPARK',
 'AUGUST ACRES',
 'CHESAPEAKE RGNL',
 'CUERO MUNI',
 'CABANISS FIELD',
 'POINT MUGU NAS (NAVAL BASE VENTURA CO)',
 'HANCOCK COUNTY-BAR HARBOR',
